In [10]:
from glob import glob
path_to_subtitles = sorted(glob("./subtitles/*.ass"))

In [12]:
path_to_subtitles[:10]

['./subtitles/001_LEG_English.ass',
 './subtitles/002_LEG_English.ass',
 './subtitles/003_LEG_English.ass',
 './subtitles/004_LEG_English.ass',
 './subtitles/005_LEG_English.ass',
 './subtitles/006_LEG_English.ass',
 './subtitles/007_LEG_English.ass',
 './subtitles/008_LEG_English.ass',
 './subtitles/009_LEG_English.ass',
 './subtitles/010_LEG_English.ass']

In [23]:
scripts=[]
episode_number=[]
for path in path_to_subtitles:
    with open(path,'r') as file:
        lines = file.readlines()
        lines = lines[27:]

        rows = [",".join(line.split(",")[9:]) for line in lines]

    rows = [line.replace("\\N"," ") for line in rows]
    script = " ".join(rows)

    episode = int(path.split('_')[0].split('/')[2].strip())
    scripts.append(script)
    episode_number.append(episode)    
    

 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220


In [26]:
import pandas as pd
df = pd.DataFrame.from_dict({'episode':episode_number,'script':scripts})


In [27]:
df.head()

,episode,script
0,1,"A long time ago, a powerful demon fox appeared..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas..."


In [106]:
import spacy

In [107]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [108]:
doc = nlp("Sasuke went to konoha")

In [109]:
for ent in doc.ents:
    print(ent.text,ent.label_)

Sasuke PERSON
konoha GPE


In [110]:
from nltk import sent_tokenize

In [111]:
def get_ners(script):
    script_sentences = sent_tokenize(script)

    ner_output = []

    for sentence in script_sentences:
        doc = nlp(sentence)
        ners = set()    
        for ent in doc.ents:
            if ent.label_ == 'PERSON':
                full_name = ent.text
                first_name = full_name.split(' ')[0]
                ners.add(first_name)
        ner_output.append(list(ners))
    return ner_output                    


In [112]:
df['ners'] = df['script'].apply(get_ners)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/suraj/nltk_data'
    - '/home/suraj/anaconda3/envs/naruto/nltk_data'
    - '/home/suraj/anaconda3/envs/naruto/share/nltk_data'
    - '/home/suraj/anaconda3/envs/naruto/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
